LLMChaning example 

In [ ]:
import os
from dotenv import load_dotenv 
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY') or ""

True

In [7]:
# print(os.getenv('OPENAI_API_KEY'))

In [2]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# from langchain.llms import ollama
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI

prompts=PromptTemplate(
    input_variables=["query"],
    template= " what is the {query}",
    
)

llm=ChatOpenAI(model="openai/gpt-4.1",base_url="https://models.github.ai/inference", temperature=0.1)
# llm=OpenAI(model="openai/o4-mini",base_url="https://openrouter.ai/api/v1", temperature=0.1)
# llm=ollama.Ollama(model="deepseek-r1:1.5b", temperature=0.9 )
chains=LLMChain(llm=llm, prompt=prompts, output_key="output", verbose=True)

C:\Users\Anupam Maiti\AppData\Local\Temp\ipykernel_13396\625507702.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chains=LLMChain(llm=llm, prompt=prompts, output_key="output", verbose=True)


In [3]:
prompt = prompts.format(query="india")
print(prompt)  # Should output: "Please tell me the capital of France"

 what is the india


In [4]:
# Step 3: Run the chain with an input
result = chains.run({"query": "capital of France"})
print(result)  # Output: "The capital of France is Paris."

C:\Users\Anupam Maiti\AppData\Local\Temp\ipykernel_13396\4189127818.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chains.run({"query": "capital of France"})




> Entering new LLMChain chain...
Prompt after formatting:
 what is the capital of France

> Finished chain.
The capital of France is Paris.


SimpleSequential Chaining of Operations**

In [5]:
capital_prompt= PromptTemplate(
    input_variables=["country"],
    template= "what is the capital {country}",
    
)
capital_chain=LLMChain(llm=llm, prompt=capital_prompt, output_key="capital", verbose=True) # Output stored as "capital"

place_prompt=PromptTemplate(
    input_variables=["capital"],
    template="what is the place where {capital} is located",
)
place_chain=LLMChain(llm=llm, prompt=place_prompt, output_key="output", verbose=True)

 

In [6]:
from langchain.chains import SimpleSequentialChain
# Create a sequential chain that first finds the capital and then finds the place
simple_chain= SimpleSequentialChain(chains=[capital_chain, place_chain],   verbose=True)
# Run the sequential chain with an input

result = simple_chain.run({"input": "India"})
print(result)  # Output: "The capital



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
what is the capital India

> Finished chain.
The capital of India is **New Delhi**.


> Entering new LLMChain chain...
Prompt after formatting:
what is the place where The capital of India is **New Delhi**. is located

> Finished chain.
The capital of India, **New Delhi**, is located in the northern part of the country. It is a district within the larger city of **Delhi**, which is officially known as the **National Capital Territory of Delhi (NCT)**. New Delhi serves as the seat of all three branches of the Government of India: the executive, the legislature, and the judiciary.

**Geographical Location:**
- **Latitude:** Approximately 28.6139° N
- **Longitude:** Approximately 77.2090° E

**Summary:**  
**New Delhi** is located within the **National Capital Territory of Delhi**, in northern India.

> Finished chain.
The capital of India, **New Delhi**, is located in the northern 

Sequential chaining  

In [8]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[capital_chain, place_chain],
    input_variables=["country"],  # Initial input
    output_variables=["capital", "output"],  # Final outputs
    verbose=True,
)

In [9]:
chain({'country':"India"})

C:\Users\Anupam Maiti\AppData\Local\Temp\ipykernel_13396\1911137727.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'country':"India"})




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
what is the capital India

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
what is the place where The capital of India is **New Delhi**. is located

> Finished chain.

> Finished chain.


{'country': 'India',
 'capital': 'The capital of India is **New Delhi**.',
 'output': 'The capital of India, **New Delhi**, is located in the **National Capital Territory of Delhi (NCT of Delhi)**, in northern India. New Delhi is a district within the larger metropolis of Delhi and serves as the seat of all three branches of the Government of India.'}

# LangChain provides several memory options:
    # ConversationBufferMemory: Stores raw chat history.
    # ConversationSummaryMemory: Stores summarized chat history (good for long conversations).
    # ConversationBufferWindowMemory: Keeps only the last k messages.
    # EntityMemory: Remembers entities (e.g., people, places) across conversations.

In [10]:
from langchain.memory import ConversationBufferMemory

country_memory = ConversationBufferMemory(input_key='country', memory_key='country_history')
capital_memory = ConversationBufferMemory(input_key='capital', memory_key='capital_history')


C:\Users\Anupam Maiti\AppData\Local\Temp\ipykernel_13396\205733047.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  country_memory = ConversationBufferMemory(input_key='country', memory_key='country_history')


In [11]:
capital_prompt= PromptTemplate(
    input_variables=["country"],
    template= "what is the capital {country}",
 
)
capital_chain=LLMChain(llm=llm, prompt=capital_prompt, output_key="capital",memory=country_memory, verbose=True) # Output stored as "capital"

place_prompt=PromptTemplate(
    input_variables=["capital"],
    template="what is the place where {capital} is located",
)
place_chain=LLMChain(llm=llm, prompt=place_prompt, output_key="output",memory=capital_memory, verbose=True)

 

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[capital_chain, place_chain],
    input_variables=["country"],  # Initial input
    output_variables=["capital", "output"],  # Final outputs
    verbose=True,
)

In [13]:
chain({'country':"India"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
what is the capital India

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
what is the place where The capital of India is **New Delhi**. is located

> Finished chain.

> Finished chain.


{'country': 'India',
 'capital': 'The capital of India is **New Delhi**.',
 'output': 'The capital of India, **New Delhi**, is located in the northern part of the country. It is a district within the larger city of **Delhi**, which is officially known as the **National Capital Territory of Delhi (NCT)**. New Delhi serves as the seat of all three branches of the Government of India: the executive, the legislature, and the judiciary.\n\n**Geographical Location:**\n- **Latitude:** Approximately 28.6139° N\n- **Longitude:** Approximately 77.2090° E\n\n**Region:** Northern India\n\n**Surrounding States:** New Delhi is bordered by the states of Haryana and Uttar Pradesh.\n\n**Summary:**  \n**New Delhi** is located within the National Capital Territory of Delhi, in northern India.'}

In [14]:
chain({'country':"ans the previous one"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
what is the capital ans the previous one

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
what is the place where It looks like your question is a bit unclear. Are you asking:

1. **What is the capital of a specific country, and what was its previous capital?**
2. **What is the capital answer and the previous one (in a quiz context)?**

Could you please clarify which country or context you are referring to? If you specify the country or context, I can provide the capital and its previous one. is located

> Finished chain.

> Finished chain.


{'country': 'ans the previous one',
 'capital': 'It looks like your question is a bit unclear. Are you asking:\n\n1. **What is the capital of a specific country, and what was its previous capital?**\n2. **What is the capital answer and the previous one (in a quiz context)?**\n\nCould you please clarify which country or context you are referring to? If you specify the country or context, I can provide the capital and its previous one.',
 'output': 'It looks like your question is incomplete or got mixed up. You wrote:\n\n> what is the place where It looks like your question is a bit unclear. Are you asking: ...\n\nCould you please clarify what you are asking? For example:\n\n- Are you asking about the location of a specific place?\n- Are you asking about the capital city of a country and its previous capital?\n- Or are you referring to something else?\n\nIf you provide more details or specify the country, city, or context, I’ll be happy to help!'}

In [15]:

country_memory.buffer

'Human: India\nAI: The capital of India is **New Delhi**.\nHuman: ans the previous one\nAI: It looks like your question is a bit unclear. Are you asking:\n\n1. **What is the capital of a specific country, and what was its previous capital?**\n2. **What is the capital answer and the previous one (in a quiz context)?**\n\nCould you please clarify which country or context you are referring to? If you specify the country or context, I can provide the capital and its previous one.'

ChatModels and ChatOpenAI

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
llm_model=ChatOpenAI(model="openai/gpt-4.1",base_url="https://models.github.ai/inference", temperature=0.1)

In [20]:
llm_model([
    SystemMessage(content="You are the helpful assistant "),
    HumanMessage(content="What is the capital of India"),
])

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 23, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'openai/gpt-4.1', 'system_fingerprint': 'fp_07e970ab25', 'finish_reason': 'stop', 'logprobs': None}, id='run--ef356ba1-12cb-4c1d-9f7b-5918a7cc8dc5-0')